In [1]:
import re
from optparse import OptionParser
from pyflann import *
import cv2
import numpy as np
import pandas as pd
import glob
import os

coordinate_header = ['FrameNumber', 'MouthHook_x', 'MouthHook_y', 'LeftMHhook_x', 'LeftMHhook_y',
                     'RightMHhook_x', 'RightMHhook_y', 'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y',
                     'RightDorsalOrgan_x', 'RightDorsalOrgan_y', 'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y',
                     'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y', 'RightBolwigOrgan_x', 'RightBolwigOrgan_y']

meta_data_header = ['FrameNumber', 'Time', 'Stage_x', 'Stage_y', 'Centroid_x', 'Centroid_y',
                    'Midpoint_x', 'Midpoint_y', 'Head_x', 'Head_y', 'Tail_x', 'Tail_y', 'MouthHook_x', 'MouthHook_y',
                    'LeftMHhook_x', 'LeftMHhook_y', 'RightMHhook_x', 'RightMHhook_y',
                    'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y', 'RightDorsalOrgan_x', 'RightDorsalOrgan_y',
                    'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y', 'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y',
                    'RightBolwigOrgan_x', 'RightBolwigOrgan_y', 'V9_x', 'V9_y', 'V10_x', 'V10_y', 'V11_x', 'V11_y',
                    'V12_x', 'V12_y', 'V13_x', 'V13_y', 'V14_x', 'V14_y', 'V15_x', 'V15_y', 'V16_x', 'V16_y',
                    'MouthHook_votes', 'LeftMHhook_votes', 'RightMHhook_votes', 'LeftDorsalOrgan_votes',
                    'RightDorsalOrgan_votes', 'CenterBolwigOrgan_votes', 'LeftBolwigOrgan_votes', 'RightBolwigOrgan_votes',
                    'V9_votes', 'V10_votes', 'V11_votes', 'V12_votes', 'V13_votes', 'V14_votes', 'V15_votes',
                    'V16_votes', 'Num_Key_points']

distance_header = ['MouthHook', 'LeftMHhook',
                   'RightMHhook', 'LeftDorsalOrgan', 'RightDorsalOrgan',
                   'CenterBolwigOrgan', 'LeftBolwigOrgan', 'RightBolwigOrgan']

meta_data_votes_header = ['MouthHook_votes', 'LeftMHhook_votes', 'RightMHhook_votes', 'LeftDorsalOrgan_votes',
                          'RightDorsalOrgan_votes', 'CenterBolwigOrgan_votes', 'LeftBolwigOrgan_votes',
                          'RightBolwigOrgan_votes', 'Num_Key_points']

roc_headers = ['true_positive', 'false_positive', 'true_negative',
               'false_negative', 'positive', 'negative',
               'true_positive_rate', 'false_positive_rate', 'total_negative',
               'total_positive', 'true_negative_rate', 'false_negative_rate',
               'precision', 'recall', 'vote_threshold']

def calcDistance(meta_data, coordinates):
    temp = (meta_data - coordinates).apply(lambda x: x**2)
    distance = pd.DataFrame([], columns=distance_header,
                            index=coordinates.index.values)
    for head in distance_header:
        not_present = np.array([])
        distance.loc[:, head] = (
            temp.loc[:, head+'_x'] + temp.loc[:, head+'_y']).apply(lambda x: np.sqrt(x))
        not_present = coordinates.loc[:, head+'_x'] == -1
        if np.where(not_present)[0].size != 0:
            distance.loc[not_present, head] = np.NaN
        distance['FrameNumber'] = coordinates.index.values
    return distance


In [2]:
crop_size = 512
confThreshold = 0
spot_size = 25.0
font = cv2.FONT_HERSHEY_SIMPLEX
test_grps_list = '../config/test_list'
video_path = '../expts/dataCollectedOn_20180417_withTestOn_20180619/videos/'
it = -1

with open(test_grps_list.strip()) as test_grps:
    for grp in test_grps:
        test_list_file_parent = os.path.split(grp)[0]
        all_test_dir = []
        pixel_to_micron = 2.75
        grp_num = str.split(grp, '_grp_')[2].split('_')[0]
        coordinates_all = pd.DataFrame([], columns=coordinate_header)
        meta_data_all = pd.DataFrame([], columns=meta_data_header)
        distance_all = pd.DataFrame([], columns=distance_header)

        with open(grp.strip()) as test_list:
            for test_file in test_list:
                test_string = str.split(os.path.split(
                    test_file.strip())[1], '_Coordinates')[0]
                video_file = os.path.join(video_path, "_".join(
                    str.split(test_string, '_')[0:3])+".avi")
                test_dir = os.path.join(test_list_file_parent, test_string)
                test_string_2 = str.split(test_string, '_')[2]
                all_test_dir.append(test_dir)
                coordinates = pd.DataFrame([])
                meta_data = pd.DataFrame([])
                distance = pd.DataFrame([])
                for fs in os.listdir(test_dir):
                    if 'Coordinates' in fs:
                        coordinates = pd.read_csv(os.path.join(
                            test_dir, fs), sep=',', names=coordinate_header)
                    if 'Metadata' in fs:
                        meta_data = pd.read_csv(os.path.join(
                            test_dir, fs), sep=',', header=0, names=meta_data_header)

                if (coordinates.empty is False) and (meta_data.empty is False):
                    coordinates = coordinates.round(0)
                    start_frame = coordinates.loc[0, 'FrameNumber'] - 1
                    coordinates.set_index('FrameNumber', inplace=True)

#                     meta_data = meta_data[meta_data.Num_Key_points != 0]
                    meta_data.loc[:, 'FrameNumber'] = meta_data.loc[:,
                                                                    'FrameNumber'] + start_frame
                    meta_data.set_index('FrameNumber', inplace=True)
                    meta_data_sel = meta_data.loc[coordinates.index,
                                                  coordinates.columns.values]
                    meta_data_sel_2 = meta_data.loc[coordinates.index, :]

                    votes = meta_data.loc[coordinates.index,
                                          meta_data_votes_header]
                    votes.columns = distance_header + ['Num_Key_points']
                    votes.reset_index(inplace=True)

                    distance = calcDistance(meta_data_sel, coordinates)
                    distance.loc[:, distance_header] = distance.loc[:,
                                                                    distance_header]

                    print "Video File: ", video_file
                    cap = cv2.VideoCapture(video_file)

                    numberFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#                     print 'Number of Video Frames Recorded : ', numberFrames

#                     outputFrameSize = np.multiply(
#                         np.ones((512, 512, 3), dtype=np.uint8), 255)
                    outputFrameSize = np.multiply(
                    np.ones((1920, 1920, 3), dtype=np.uint8), 255)
                    height, width, layers = outputFrameSize.shape

                    fourcc = cv2.VideoWriter_fourcc(*'XVID')
                    outputVideoFile = os.path.join(test_dir,
                                                   '_'.join(str.split(test_string, '_')[0:3]) + '.avi')
                    print "Annotated File : ", outputVideoFile
                    da = '../expts/dataCollectedOn_20180417_withTestOn_20180619/videos/Rawdata_20180417_082627.avi'
#                     if video_file == da:
#                         break
                    video = cv2.VideoWriter(
                        outputVideoFile, 0, 5, (width, height))

                    if cap.isOpened():
                        for row_index, row in meta_data_sel_2.iterrows():
                            cap.set(1, row_index)
                            ret, originalFrame = cap.read()

                            frame = originalFrame.copy()
                            frameOverlay = originalFrame.copy()

                            cropCenter_X = row['Head_x']
                            cropCenter_Y = row['Head_y']
                            if np.isnan(cropCenter_X) or np.isnan(cropCenter_Y):
                                cropCenter_X = coordinates.loc[row_index,
                                                               'MouthHook_x']
                                cropCenter_Y = coordinates.loc[row_index,
                                                               'MouthHook_y']

                            crop_x = int(max(0, cropCenter_X-int(crop_size/2)))
                            crop_y = int(max(0, cropCenter_Y-int(crop_size/2)))
                            
                            cv2.putText(frameOverlay, '%d'%(row_index),
                                        (crop_x + 360, crop_y + 40),
                                        font, 0.65, (0, 0, 0), 2, cv2.LINE_AA)
                            cv2.putText(frameOverlay, 'Spot - %d um' % (spot_size*2.75),
                                        (crop_x + 360, crop_y + 120),
                                        font, 0.65, (0, 0, 0), 2, cv2.LINE_AA)

                            for bp in ['MouthHook', 'LeftDorsalOrgan', 'RightDorsalOrgan']:
                                bp_x = row[bp+'_x']
                                bp_y = row[bp+'_y']
                                bp_votes = row[bp+'_votes']
                                bp_kp = row['Num_Key_points']

                                bp_c_x = int(
                                    coordinates.loc[row_index, bp+'_x'])
                                bp_c_y = int(
                                    coordinates.loc[row_index, bp+'_y'])
                                
                                if (not np.isnan(bp_x)) and (not np.isnan(bp_y)):
                                    recTopLeft = {}
                                    recTopLeft['x'] = int(
                                        bp_x) - int(spot_size/2)
                                    recTopLeft['y'] = int(
                                        bp_y) - int(spot_size/2)

                                    recBottomRight = {}
                                    recBottomRight['x'] = int(
                                        bp_x) + int(spot_size/2)
                                    recBottomRight['y'] = int(
                                        bp_y) + int(spot_size/2)
                                    
                                    cv2.putText(frame, '%d'%(int(bp_kp)),
                                                (crop_x + 440, crop_y + 40),
                                                font, 0.65, (0, 0, 0), 2, cv2.LINE_AA)
                                    
                                    cv2.circle(frameOverlay, (int(cropCenter_X), int(cropCenter_Y)),
                                               radius=7, color=(0, 0, 0), thickness=-1)
                                    
                                    if (bp == 'LeftDorsalOrgan') and (bp_votes >= 0):
                                        cv2.rectangle(frameOverlay,
                                                      (recTopLeft['x'],
                                                       recTopLeft['y']),
                                                      (recBottomRight['x'],
                                                       recBottomRight['y']),
                                                      color=(125, 125, 255), thickness=-1)
                                        cv2.circle(frameOverlay, (bp_c_x, bp_c_y),
                                                   radius=5, color=(0, 0, 255), thickness=-1)
                                        cv2.putText(frame, '%s - %d' % ('LDO', bp_votes),
                                                    (crop_x + 360, crop_y + 80),
                                                    font, 0.65, (125, 125, 255), 2, cv2.LINE_AA)
                                    elif (bp == 'RightDorsalOrgan') and (bp_votes >= 0):
                                        cv2.rectangle(frameOverlay,
                                                      (recTopLeft['x'],
                                                       recTopLeft['y']),
                                                      (recBottomRight['x'],
                                                       recBottomRight['y']),
                                                      color=(0, 255, 0), thickness=-1)
                                        cv2.circle(frameOverlay, (bp_c_x, bp_c_y),
                                                   radius=5, color=(0, 255, 0), thickness=-1)
                                        cv2.putText(frame, '%s - %d' % ('RDO', bp_votes),
                                                    (crop_x + 360, crop_y + 100),
                                                    font, 0.65, (0, 255, 0), 2, cv2.LINE_AA)
                                    elif (bp == 'MouthHook') and (bp_votes >= 0):
                                        cv2.rectangle(frameOverlay,
                                                      (recTopLeft['x'],
                                                       recTopLeft['y']),
                                                      (recBottomRight['x'],
                                                       recBottomRight['y']),
                                                      color=(255, 0, 0), thickness=-1)
                                        cv2.circle(frameOverlay, (bp_c_x, bp_c_y),
                                                   radius=5, color=(255, 0, 0), thickness=-1)
                                        cv2.putText(frame, '%s - %d' % ('MH', bp_votes),
                                                    (crop_x + 360, crop_y + 60),
                                                    font, 0.65, (255, 0, 0), 2, cv2.LINE_AA)

                                elif (not np.isnan(bp_c_x)) and (not np.isnan(bp_c_y)):
                                    if (bp == 'LeftDorsalOrgan'):
                                        cv2.circle(frameOverlay, (bp_c_x, bp_c_y),
                                                   radius=5, color=(0, 0, 255), thickness=-1)
                                        cv2.putText(frame, '%s - %d' % ('LDO', 0),
                                                    (crop_x + 360, crop_y + 80),
                                                    font, 0.65, (125, 125, 255), 2, cv2.LINE_AA)                                
                                    elif (bp == 'RightDorsalOrgan'):
                                        cv2.circle(frameOverlay, (bp_c_x, bp_c_y),
                                                   radius=5, color=(0, 255, 0), thickness=-1)
                                        cv2.putText(frame, '%s - %d' % ('RDO', 0),
                                                    (crop_x + 360, crop_y + 100),
                                                    font, 0.65, (0, 255, 0), 2, cv2.LINE_AA)    
                                    elif (bp == 'MouthHook'):
                                        cv2.circle(frameOverlay, (bp_c_x, bp_c_y),
                                                   radius=5, color=(255, 0, 0), thickness=-1)
                                        cv2.putText(frame, '%s - %d' % ('MH', 0),
                                                    (crop_x + 360, crop_y + 60),
                                                    font, 0.65, (255, 0, 0), 2, cv2.LINE_AA)

#                             frame = frame[crop_y:crop_y+crop_size,
#                                           crop_x:crop_x+crop_size, :]
#                             frameOverlay = frameOverlay[crop_y:crop_y +
#                                                         crop_size, crop_x:crop_x+crop_size, :]
                            
                            
                            alpha = 0.3
                            cv2.addWeighted(frameOverlay, alpha,
                                            frame, 1 - alpha, 0, frame)
                            video.write(frame)

                    cv2.destroyAllWindows()
                    video.release()

Video File:  ../expts/dataCollectedOn_20180417_withTestOn_20180619/videos/Rawdata_20180417_083618.avi
Annotated File :  ../expts/dataCollectedOn_20180417_withTestOn_20180619/dataCollectedOn_2018041_grp_1/20180529_150721/Rawdata_20180417_083618_20180420_104633/Rawdata_20180417_083618.avi
Video File:  ../expts/dataCollectedOn_20180417_withTestOn_20180619/videos/Rawdata_20180417_084502.avi
Annotated File :  ../expts/dataCollectedOn_20180417_withTestOn_20180619/dataCollectedOn_2018041_grp_1/20180529_150721/Rawdata_20180417_084502_20180423_075225/Rawdata_20180417_084502.avi
Video File:  ../expts/dataCollectedOn_20180417_withTestOn_20180619/videos/Rawdata_20180417_082627.avi
Annotated File :  ../expts/dataCollectedOn_20180417_withTestOn_20180619/dataCollectedOn_2018041_grp_2/20180529_150807/Rawdata_20180417_082627_20180525_112343_A_133410/Rawdata_20180417_082627.avi
Video File:  ../expts/dataCollectedOn_20180417_withTestOn_20180619/videos/Rawdata_20180417_084502.avi
Annotated File :  ../expt

In [3]:
# import seaborn as sns
# sns.distplot(meta_data_sel_2['Num_Key_points'].dropna(), kde=False, color="b")
# # sns.distplot(meta_data_sel_2['Num_Key_points'], hist=False, color="g", kde_kws={"shade": True})